In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import math

import sklearn_relief as relief

%matplotlib inline

In [2]:
df = pd.read_excel("data/baza_HR_v5.xlsx")
df.head()

,Numbers,ageattest,Gender,CR_Code,ethnic_code,height inch->cm,weight Ib->kg,BMI,waist,bodyfat,...,W_step3_F_LT,W_step3_F_GT,W_step3_F_EQ,W_step3_M,W_step3_F,W_step4_M,W_step4_F,Wasserman_vo2_l_kg_min,HRmax1,HRmax2
0,2570,42.0,Male,2.0,0.0,186.70,84.81,24.330914,84.40,14.610523,...,1950.034300,1938.650200,1998.9484,3011.289444,1938.650200,3342.531283,2151.901722,39.411995,178.0,178.96
1,1,55.0,Female,2.0,4.0,159.99,87.79,34.297256,NaN,NaN,...,1577.914203,1558.897705,1756.5097,2120.430346,1558.897705,2120.430346,1558.897705,17.757122,165.0,169.60
2,2,62.0,Female,2.0,4.0,157.00,78.09,31.680798,NaN,44.800000,...,1366.840800,1364.580000,1482.1416,1840.014480,1364.580000,1840.014480,1364.580000,17.474453,158.0,164.56
3,3,36.0,Male,3.0,NaN,172.72,73.75,24.721611,NaN,NaN,...,1909.385940,1899.408880,1945.0550,2790.245603,1899.408880,2790.245603,1899.408880,37.833839,184.0,183.28
4,4,58.0,Male,2.0,0.0,185.42,80.82,23.507459,93.01,23.100000,...,1579.747780,1578.323160,1599.7544,2427.721430,1578.323160,2427.721430,1578.323160,30.038622,162.0,167.44


In [3]:
# get rid of current predictions ... we do not need those ...
df_clear = df.drop(['VO2_Mayers', 'W_step1_M_CF', 'W_step1_F_CF', 'W_step1_M_PW', 'W_step1_F_PW', 'W_step2_M', 'W_step2_F',
'W_step3_M_LT', 'W_step3_M_GT', 'W_step3_M_EQ', 'W_step3_F_LT', 'W_step3_F_GT', 'W_step3_F_EQ',  'W_step3_M',
'W_step3_F', 'W_step4_M', 'W_step4_F', 'Wasserman_vo2_l_kg_min', 'HRmax1', 'HRmax2'],axis=1)
df_clear.head()

,Numbers,ageattest,Gender,CR_Code,ethnic_code,height inch->cm,weight Ib->kg,BMI,waist,bodyfat,...,vo2_ml_kg_min,ve_btbs,peak_rpe,OUES,OscillatoryBreathing,test_termination_reason,Symptoms,gxt_st_changes,gxt_arr,follow_up
0,2570,42.0,Male,2.0,0.0,186.70,84.81,24.330914,84.40,14.610523,...,47.8,203.60,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,55.0,Female,2.0,4.0,159.99,87.79,34.297256,NaN,NaN,...,45.6,94.24,18.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,62.0,Female,2.0,4.0,157.00,78.09,31.680798,NaN,44.800000,...,29.0,95.00,NaN,2.84,NaN,NaN,NaN,NaN,NaN,NaN
3,3,36.0,Male,3.0,NaN,172.72,73.75,24.721611,NaN,NaN,...,13.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,58.0,Male,2.0,0.0,185.42,80.82,23.507459,93.01,23.100000,...,22.9,74.00,NaN,2.01,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
to_dummies = ['CR_Code', 'ethnic_code', 'Gender', 'PAStatus', 'Smoked ', 
              'educlevel', 'employment_status', 'marital_status', 
              'DiureticMed', 'PackDay', 'metabolic_system', 
              'protocol', 'Symptoms', 'test_termination_reason', 
              'TestIndication']

df_dummies = df_clear

for i in to_dummies:
    dummies = pd.get_dummies(df_clear[i], drop_first=True, dummy_na=True, prefix=i)
    start_with = str(i + '_')
    dummies.loc[df_dummies[i].isnull(), dummies.columns.str.startswith(start_with)] = np.nan
    dummies = dummies.drop(str(i + '_nan'), axis=1)
    df_dummies = df_dummies.drop(i, axis=1)
    df_dummies = pd.concat([df_dummies, dummies], axis=1)

In [5]:
# '@1rsSmoked', 'PackDay_1.0', 'PackDay_3.0' represent 2% or less ... so we will
# remove these attributes ...
# in addition, we will also concat 'Smoked _1.0', 'Smoked _2.0' into one
# variable, smoked or not ...
testing = df_dummies[['@1rsSmoked', 'PackDay_1.0', 'PackDay_3.0', 'Smoked _1.0', 'Smoked _2.0']]
testing['Smoked'] = df_clear.loc[:,'Smoked ']
testing[testing['Smoked'] == 2] = 1
testing = testing.drop(['@1rsSmoked', 'PackDay_1.0', 'PackDay_3.0', 'Smoked _1.0', 'Smoked _2.0'], axis=1)
df_dummies = df_dummies.drop(['@1rsSmoked', 'PackDay_1.0', 'PackDay_3.0', 'Smoked _1.0', 'Smoked _2.0'], axis=1)

<ipython-input-5-4d76eabbb0f1>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  testing['Smoked'] = df_clear.loc[:,'Smoked ']
<ipython-input-5-4d76eabbb0f1>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  testing[testing['Smoked'] == 2] = 1
C:\Users\Larsen\anaconda3\lib\site-packages\pandas\core\frame.py:2957: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html

In [6]:
testing = pd.concat([testing, df_dummies[['PhysInactive', 'PAStatus_1.0', 'PAStatus_2.0', 'PAStatus_3.0', 'PAStatus_4.0', 'PAStatus_5.0', 'PAStatus_6.0']]], axis=1)
# it turns out, the best way to merge these attributes is by saying
# that PAStatus 5 and 6 are Physically Inactive people --> PhysInactive = 1
# PAStatus 1,2,3,4 are Physically Active people --> PhysInactive = 0
testing[testing['PAStatus_6.0'] == 1]['PhysInactive'] = 1
testing[testing['PAStatus_5.0'] == 1]['PhysInactive'] = 1
testing[testing['PAStatus_1.0'] == 1]['PhysInactive'] = 0
testing[testing['PAStatus_2.0'] == 1]['PhysInactive'] = 0
testing[testing['PAStatus_3.0'] == 1]['PhysInactive'] = 0
testing[testing['PAStatus_4.0'] == 1]['PhysInactive'] = 0
testing = testing.drop(['PAStatus_1.0', 'PAStatus_2.0', 'PAStatus_3.0', 'PAStatus_4.0', 'PAStatus_5.0', 'PAStatus_6.0'], axis=1)
df_dummies = df_dummies.drop(['PhysInactive', 'PAStatus_1.0', 'PAStatus_2.0', 'PAStatus_3.0', 'PAStatus_4.0', 'PAStatus_5.0', 'PAStatus_6.0'], axis=1)

<ipython-input-6-296a6177671a>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  testing[testing['PAStatus_6.0'] == 1]['PhysInactive'] = 1
<ipython-input-6-296a6177671a>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  testing[testing['PAStatus_5.0'] == 1]['PhysInactive'] = 1
<ipython-input-6-296a6177671a>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.

In [7]:
# both have to be 1!
testing = pd.concat([testing, df_dummies[['resting_ecg', 'gxt_arr']]], axis=1)
testing['Aritmia'] = testing['resting_ecg'] + testing['gxt_arr']
testing[testing['Aritmia'] == 1] = 0
testing[testing['Aritmia'] == 2] = 1
testing = testing.drop(['resting_ecg', 'gxt_arr'], axis=1)
df_dummies = df_dummies.drop(['resting_ecg', 'gxt_arr'], axis=1)

In [8]:
testing = pd.concat([testing, df_dummies], axis=1)

In [9]:
testing.head()

,Smoked,PhysInactive,Aritmia,Numbers,ageattest,height inch->cm,weight Ib->kg,BMI,waist,bodyfat,...,test_termination_reason_2.0,test_termination_reason_3.0,test_termination_reason_4.0,test_termination_reason_5.0,test_termination_reason_6.0,test_termination_reason_7.0,test_termination_reason_8.0,TestIndication_2.0,TestIndication_3.0,TestIndication_4.0
0,NaN,1.0,NaN,2570,42.0,186.70,84.81,24.330914,84.40,14.610523,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0
1,0.0,1.0,NaN,1,55.0,159.99,87.79,34.297256,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0
2,0.0,0.0,NaN,2,62.0,157.00,78.09,31.680798,NaN,44.800000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0
3,NaN,NaN,NaN,3,36.0,172.72,73.75,24.721611,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0
4,1.0,1.0,NaN,4,58.0,185.42,80.82,23.507459,93.01,23.100000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0


In [10]:
# Basic ... mean and median ...
for col in testing:
    print(col)
    if col == 'Numbers':
        continue
        
    d = testing[col].describe()
    replace = 0
    # print(d.loc['max'])
    if (d.loc['max'] == 1):
        # we have a binary attribute, so take median
        replace = d.loc['50%']
    else:
    # else we have continuous variable, so we take mean
        replace = round(d.loc['mean'], 2)
    print(replace)
    testing[col] = testing[col].fillna(replace)

Smoked
0.0
PhysInactive
0.0
Aritmia
0.0
Numbers
ageattest
46.06
height inch->cm
175.49
weight Ib->kg
83.63
BMI
27.04
waist
90.26
bodyfat
26.28
AnyMeds AN do AY 
0.0
All_disease
0.0
Mode
1.0
resting_hr
72.15
resting_sbp
121.83
resting_dbp
78.75
ETtime
10.52
PETC02_peak
38.44
max_hr
165.89
peak_sbp
178.78
peak_dbp
81.74
ve_vc02_slope
24.91
rec_hr1_dif
27.57
RecHr2min_dif
35.52
fvc
4.45
fev1
3.53
acearb
0.0
AntiarrMed
0.0
AnticorMed
0.0
AntihyptMed
0.0
CalcChMed
0.0
DigoxinMed
0.0
EstrogMed
0.0
NitrateMed
0.0
StatinMed
0.0
OtherMeds
0.0
NoknownDis
0.0
HeartFailure
0.0
Cancer
0.0
CKD
0.0
Diabetes
0.0
Endocrine
0.0
Liver
0.0
Neurological
0.0
Asthma
0.0
OtherDisease
0.0
Hypertension
0.0
Hyperlipidem
0.0
CADFamHist
0.0
ModeCode
0.0
PETC02_rest
34.26
RestO2Sat
96.79
VT
15.79
HR_at_VT
112.05
speed
4.99
grade
10.73
max_load_watts
190.92
max_rer
1.15
vo2_l_min
2.61
vo2_ml_kg_min
31.91
ve_btbs
84.18
peak_rpe
17.55
OUES
2.46
OscillatoryBreathing
0.0
gxt_st_changes
0.0
follow_up
0.0
CR_Code_1.0
0.0


In [11]:
testing.head()

,Smoked,PhysInactive,Aritmia,Numbers,ageattest,height inch->cm,weight Ib->kg,BMI,waist,bodyfat,...,test_termination_reason_2.0,test_termination_reason_3.0,test_termination_reason_4.0,test_termination_reason_5.0,test_termination_reason_6.0,test_termination_reason_7.0,test_termination_reason_8.0,TestIndication_2.0,TestIndication_3.0,TestIndication_4.0
0,0.0,1.0,0.0,2570,42.0,186.70,84.81,24.330914,84.40,14.610523,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,1.0,0.0,1,55.0,159.99,87.79,34.297256,90.26,26.280000,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,2,62.0,157.00,78.09,31.680798,90.26,44.800000,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,3,36.0,172.72,73.75,24.721611,90.26,26.280000,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,1.0,0.0,4,58.0,185.42,80.82,23.507459,93.01,23.100000,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
leave_out = ['Numbers', 'max_hr', 'vo2_l_min', 'vo2_ml_kg_min']
mins = testing.drop(leave_out, axis=1).min(axis=0)
maxs = testing.drop(leave_out, axis=1).max(axis=0)

In [13]:
mins

Smoked                           0.00
PhysInactive                     0.00
Aritmia                          0.00
ageattest                       19.50
height inch->cm                143.51
                                ...  
test_termination_reason_7.0      0.00
test_termination_reason_8.0      0.00
TestIndication_2.0               0.00
TestIndication_3.0               0.00
TestIndication_4.0               0.00
Length: 112, dtype: float64

In [14]:
maxs

Smoked                           1.00
PhysInactive                     1.00
Aritmia                          1.00
ageattest                       89.20
height inch->cm                208.28
                                ...  
test_termination_reason_7.0      1.00
test_termination_reason_8.0      1.00
TestIndication_2.0               1.00
TestIndication_3.0               1.00
TestIndication_4.0               1.00
Length: 112, dtype: float64

In [15]:
testing_scaled = (testing.drop(leave_out, axis=1) - testing.drop(leave_out, axis=1).min(axis=0)) / (testing.drop(leave_out, axis=1).max(axis=0) - testing.drop(leave_out, axis=1).min(axis=0))
testing_scaled = pd.concat([testing_scaled, testing[leave_out]], axis=1)

In [16]:
variances = ((testing_scaled.drop(leave_out, axis=1) - testing_scaled.drop(leave_out, axis=1).mean(axis=0))**2).sum(axis=0) / len(testing_scaled.drop(leave_out, axis=1).index)

In [17]:
variances.sort_values()

CKD                     0.000047
DiureticMed_2.0         0.000047
Liver                   0.000093
TestIndication_2.0      0.000093
educlevel_Elementary    0.000093
                          ...   
Gender_Male             0.185383
CR_Code_1.0             0.200049
CR_Code_2.0             0.216734
ModeCode                0.247351
Mode                    0.247366
Length: 112, dtype: float64

In [18]:
pip install sklearn-relief==1.0.0b2

Note: you may need to restart the kernel to use updated packages.


In [19]:
df_final = testing_scaled.drop('Numbers', axis=1).dropna()

In [20]:
# EXPORT
# df_final.to_csv('all_normalized.csv')
# mins.to_csv('mins_all.csv')
# maxs.to_csv('maxs_all.csv')

In [43]:
def test_cv_features(X, y, k=5, best=5):
    N = len(y)
    first = N % k
    first_n = int(N / k) + 1
    n = int(N / k)

    # print(N, first, first_n, n)

    shape = X.shape
    shuffle = np.zeros((shape[0], shape[1] + 1))
    print(shape) # index 1 num of columns
    shuffle[:, :-1] = X
    shuffle[:, -1] = y

    # np.random.shuffle(shuffle)
    per = np.random.permutation(N)

    feature_scores = np.zeros(shape[1])

    for i in range(0, k):
        print(i+1, "iteration")
        
        if i < first:
            from_i = i * first_n
            to_i = (i + 1) * first_n
            # print(i * first_n, (i + 1) * first_n)
        else:
            from_i = (first * first_n) + (i - first) * n
            to_i = (first * first_n) + (i - first + 1) * n
            # print((first * first_n) + (i - first) * n, (first * first_n) + (i - first + 1) * n)

        Xi_train = np.delete(shuffle, np.s_[per[from_i:to_i]], 0)
        Xi_test = shuffle[per[from_i:to_i]]
        
        attr_vector = np.array(Xi_train[:, :-1])
        target_vectr = np.array(Xi_train[:, -1])
        # print(attr_vector)
        # print(target_vectr)
        
        relief_features = relief.Relief(
            n_features=best # Choose the best 5 features
        ) # Will run by default on all processors concurrently

        matrix = relief_features.fit_transform(
            attr_vector,
            target_vectr
        )
        
        # print(relief_features.w_)
        
        feature_scores = feature_scores + relief_features.w_
        # print(feature_scores)
                               
    return (feature_scores / k)

In [22]:
target = ['max_hr', 'vo2_l_min', 'vo2_ml_kg_min']
df_final.columns

Index(['Smoked', 'PhysInactive', 'Aritmia', 'ageattest', 'height inch->cm',
       'weight Ib->kg', 'BMI', 'waist', 'bodyfat', 'AnyMeds AN do AY ',
       ...
       'test_termination_reason_5.0', 'test_termination_reason_6.0',
       'test_termination_reason_7.0', 'test_termination_reason_8.0',
       'TestIndication_2.0', 'TestIndication_3.0', 'TestIndication_4.0',
       'max_hr', 'vo2_l_min', 'vo2_ml_kg_min'],
      dtype='object', length=115)

## HR max 10-fold CV feature selection

5 nearest

In [60]:
feat_ranking = test_cv_features(df_final.drop(target, axis=1), df_final['max_hr'], 10, 5)
r = list(feat_ranking) 
c = df_final.columns.drop(target)

attr = [x for _, x in sorted(zip(r,c))]
r = np.sort(r)
for i in range(0, len(attr)):
  print("{a:25s} --> {p:3f}".format(a = attr[i], p = r[i]))

(21432, 112)
1 iteration
2 iteration
3 iteration
4 iteration
5 iteration
6 iteration
7 iteration
8 iteration
9 iteration
10 iteration
AntiarrMed                --> 0.000000
Aritmia                   --> 0.000000
CADFamHist                --> 0.000000
CKD                       --> 0.000000
CR_Code_4.0               --> 0.000000
CR_Code_5.0               --> 0.000000
Cancer                    --> 0.000000
Diabetes                  --> 0.000000
DigoxinMed                --> 0.000000
DiureticMed_2.0           --> 0.000000
Mode                      --> 0.000000
ModeCode                  --> 0.000000
Neurological              --> 0.000000
OscillatoryBreathing      --> 0.000000
OtherDisease              --> 0.000000
Symptoms_1.0              --> 0.000000
Symptoms_3.0              --> 0.000000
TestIndication_2.0        --> 0.000000
TestIndication_3.0        --> 0.000000
TestIndication_4.0        --> 0.000000
educlevel_Elementary      --> 0.000000
employment_status_Student --> 0.000000
ethnic_c

10 nearest

In [61]:
feat_ranking = test_cv_features(df_final.drop(target, axis=1), df_final['max_hr'], 10, 10)
r = list(feat_ranking) 
c = df_final.columns.drop(target)

attr = [x for _, x in sorted(zip(r,c))]
r = np.sort(r)
for i in range(0, len(attr)):
  print("{a:25s} --> {p:3f}".format(a = attr[i], p = r[i]))

(21432, 112)
1 iteration
2 iteration
3 iteration
4 iteration
5 iteration
6 iteration
7 iteration
8 iteration
9 iteration
10 iteration
AntiarrMed                --> 0.000000
Asthma                    --> 0.000000
CKD                       --> 0.000000
CR_Code_2.0               --> 0.000000
CR_Code_4.0               --> 0.000000
Cancer                    --> 0.000000
Diabetes                  --> 0.000000
DiureticMed_2.0           --> 0.000000
HeartFailure              --> 0.000000
Mode                      --> 0.000000
ModeCode                  --> 0.000000
Neurological              --> 0.000000
NoknownDis                --> 0.000000
OscillatoryBreathing      --> 0.000000
OtherDisease              --> 0.000000
Symptoms_1.0              --> 0.000000
Symptoms_2.0              --> 0.000000
Symptoms_3.0              --> 0.000000
TestIndication_2.0        --> 0.000000
TestIndication_3.0        --> 0.000000
TestIndication_4.0        --> 0.000000
educlevel_Elementary      --> 0.000000
employme

all

In [62]:
feat_ranking = test_cv_features(df_final.drop(target, axis=1), df_final['max_hr'], 10, 112)
r = list(feat_ranking) 
c = df_final.columns.drop(target)

attr = [x for _, x in sorted(zip(r,c))]
r = np.sort(r)
for i in range(0, len(attr)):
  print("{a:25s} --> {p:3f}".format(a = attr[i], p = r[i]))

(21432, 112)
1 iteration
2 iteration
3 iteration
4 iteration
5 iteration
6 iteration
7 iteration
8 iteration
9 iteration
10 iteration
Asthma                    --> 0.000000
CADFamHist                --> 0.000000
CKD                       --> 0.000000
CR_Code_4.0               --> 0.000000
Diabetes                  --> 0.000000
DiureticMed_2.0           --> 0.000000
Endocrine                 --> 0.000000
Liver                     --> 0.000000
Mode                      --> 0.000000
ModeCode                  --> 0.000000
NoknownDis                --> 0.000000
OscillatoryBreathing      --> 0.000000
Symptoms_2.0              --> 0.000000
Symptoms_3.0              --> 0.000000
TestIndication_2.0        --> 0.000000
TestIndication_3.0        --> 0.000000
TestIndication_4.0        --> 0.000000
employment_status_Student --> 0.000000
marital_status_Separated  --> 0.000000
marital_status_Widowed    --> 0.000000
metabolic_system_2.0      --> 0.000000
metabolic_system_4.0      --> 0.000000
metaboli

## VO2 max 10-fold CV feature selection

10 nearest

In [64]:
feat_ranking = test_cv_features(df_final.drop(target, axis=1), df_final['vo2_ml_kg_min'], 10, 10)
r = list(feat_ranking) 
c = df_final.columns.drop(target)

attr = [x for _, x in sorted(zip(r,c))]
r = np.sort(r)
for i in range(0, len(attr)):
  print("{a:25s} --> {p:3f}".format(a = attr[i], p = r[i]))

(21432, 112)
1 iteration
2 iteration
3 iteration
4 iteration
5 iteration
6 iteration
7 iteration
8 iteration
9 iteration
10 iteration
Asthma                    --> 0.000000
CKD                       --> 0.000000
CR_Code_4.0               --> 0.000000
Diabetes                  --> 0.000000
DiureticMed_2.0           --> 0.000000
HeartFailure              --> 0.000000
Liver                     --> 0.000000
Mode                      --> 0.000000
ModeCode                  --> 0.000000
NoknownDis                --> 0.000000
OscillatoryBreathing      --> 0.000000
Symptoms_2.0              --> 0.000000
Symptoms_3.0              --> 0.000000
Symptoms_5.0              --> 0.000000
TestIndication_2.0        --> 0.000000
TestIndication_3.0        --> 0.000000
TestIndication_4.0        --> 0.000000
educlevel_Elementary      --> 0.000000
ethnic_code_1.0           --> 0.000000
ethnic_code_5.0           --> 0.000000
follow_up                 --> 0.000000
gxt_st_changes            --> 0.000000
marital_

all

In [63]:
feat_ranking = test_cv_features(df_final.drop(target, axis=1), df_final['vo2_ml_kg_min'], 10, 112)
r = list(feat_ranking) 
c = df_final.columns.drop(target)

attr = [x for _, x in sorted(zip(r,c))]
r = np.sort(r)
for i in range(0, len(attr)):
  print("{a:25s} --> {p:3f}".format(a = attr[i], p = r[i]))

(21432, 112)
1 iteration
2 iteration
3 iteration
4 iteration
5 iteration
6 iteration
7 iteration
8 iteration
9 iteration
10 iteration
CKD                       --> 0.000000
CR_Code_4.0               --> 0.000000
CR_Code_5.0               --> 0.000000
Cancer                    --> 0.000000
Diabetes                  --> 0.000000
DigoxinMed                --> 0.000000
DiureticMed_2.0           --> 0.000000
Endocrine                 --> 0.000000
Liver                     --> 0.000000
Mode                      --> 0.000000
ModeCode                  --> 0.000000
OscillatoryBreathing      --> 0.000000
OtherDisease              --> 0.000000
Symptoms_1.0              --> 0.000000
Symptoms_2.0              --> 0.000000
Symptoms_3.0              --> 0.000000
Symptoms_4.0              --> 0.000000
Symptoms_5.0              --> 0.000000
TestIndication_2.0        --> 0.000000
TestIndication_3.0        --> 0.000000
TestIndication_4.0        --> 0.000000
educlevel_Elementary      --> 0.000000
employme